Example Notebook
============

In [28]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
basedir = '../'
sys.path.append(basedir)

import numpy as np
import pandas as pd
from IPython.display import display
from lda_for_fragments import Ms2Lda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h2>1. Feature Extraction</h2>

There are two ways to generate the input matrices, described below in option (a) and (b).

<h3>a. Loading Existing Input Matrices</h3>

You can run the R feature extraction pipeline separately to produce the necessary input matrices for MS2LDA. All the R scripts necessary to perform feature extraction can be found in the "R" folder (one level above from the current "notebooks" folder you're in now). 

The entry point to the pipeline is **R/MS1MS2_MatrixGeneration.R**, so load that file in your favourite R development environment (e.g. RStudio), set the working directory to the "R" folder. Next, you'd need to configure all the parameters of the pipeline. This can be found in **config.yml** file. The default parameters provided there were were for our data generated on pHILIC-MS and HILIC-MS runs in positive ionisation mode. You can try with these values first and modify them as necessary if you aren't getting sensible results in the count matrices that can be used by LDA.

Note: RMassBank is one of the dependencies of the pipeline. RMassBank relies on rJava. The following is a common problem that you might encounter when configuring rJava : http://stackoverflow.com/questions/12872699/error-unable-to-load-installed-packages-just-now. 

In the cell below, we load some CSV files produced after running the feature extraction pipeline in R.

In [29]:
fragment_filename = basedir + 'input/final/Beer_3_full1_5_2E5_pos_fragments.csv'
neutral_loss_filename = basedir + 'input/final/Beer_3_full1_5_2E5_pos_losses.csv'
mzdiff_filename = None
ms1_filename = basedir + 'input/final/Beer_3_full1_5_2E5_pos_ms1.csv'
ms2_filename = basedir + 'input/final/Beer_3_full1_5_2E5_pos_ms2.csv'
ms2lda = Ms2Lda.lcms_data_from_R(fragment_filename, neutral_loss_filename, mzdiff_filename, 
                             ms1_filename, ms2_filename)

Loading input files


FileNotFoundError: File b'../input/final/Beer_3_full1_5_2E5_pos_fragments.csv' does not exist

<h3>b. Running the Feature Extraction Pipeline</h3>

For convenience, we've also provided a wrapper method (shown below) in MS2-LDA to call the the feature extraction pipeline. This takes as input the full scan and fragmentation files (defined in config_filename) and produces the various count matrices used as input to LDA.

Note: Since we're calling R codes from Python, this depends on http://rpy.sourceforge.net/, which doesn't seem to be well-supported in Windows.

In [ ]:
# path to the folder containing the R scripts used for feature extraction 
script_folder = '/home/joewandy/git/metabolomics_tools/justin/R'

# path to the configuration file for feature extraction
config_filename = os.path.join(script_folder, 'config.yml')

# too many warning messages printed from R
import warnings
warnings.filterwarnings("ignore")

# run the feature extraction pipeline, this will run for a long time!!
ms2lda = Ms2Lda.run_feature_extraction(script_folder, config_filename)

<hr/>

<h2>2. Analysis</h2>

<h3>a. Run LDA</h3>

Once the data has been loaded by performing either step 1(a) or 1(b), we're now ready to run LDA analysis.

In [ ]:
### all the parameters you need to specify to run LDA ###

n_topics = 300 # 300 - 400 topics from cross-validation
n_samples = 1000 # 100 is probably okay for testing. For manuscript, use > 500-1000.
n_burn = 0 # if 0 then we only use the last sample
n_thin = 1 # every n-th sample to use for averaging after burn-in. Ignored if n_burn = 0
alpha = 50.0/n_topics # hyper-parameter for document-topic distributions
beta = 0.1 # hyper-parameter for topic-word distributions

ms2lda.run_lda(n_topics, n_samples, n_burn, n_thin, alpha, beta)

<h3>b. (Optional) In-silico Annotation using SIRIUS</h3>

For the purpose of visualisation in step 3(c), we can annotate the MS1 and MS2 peaks using [SIRIUS](http://bio.informatik.uni-jena.de/software/sirius/), an in-silico fragmentation tool written in Java. At the moment, each parent MS1 peak and its associated MS2 spectra are run through SIRIUS separately. Isotopic information, which can be used to improve annotation, is not used yet.

If you run this annotation step before saving the project in step (c) below, the annotation information will be saved into the ms1 and ms2 peak info too.

In [ ]:
# sirius_platform specifies the profile used by SIRIUS
# ppm_max is the mass tolerance used by SIRIUS when assigning elemental formulae
# mode is either 'pos' or 'neg'
# max_ms1 excludes any MS1 with m/z > 400 from annotation since it takes too long to process
ms2lda.annotate_with_sirius(sirius_platform='orbitrap', ppm_max=5, mode='pos', max_ms1=400)

In [ ]:
display(ms2lda.ms1)

In [ ]:
display(ms2lda.ms2)

<h3>c. (Optional) In-silico Annotation using EF Assigner</h3>

If you have not installed SIRIUS, we have also provided a simple in-silico annotation method (EF Assigner) written completely in Python. The method works by combinatorially enumerating all candidate formulae that can be produced by the precursor mass, applying the 7 golden rules to reduce the candidate set and returning the formula closest in mass to the observed precursor mass as the 'top hit'. This method does not assign formulae to the losses, only the fragments.

Similar to the SIRIUS annotation above, we can call EF Assigner as shown in the cells below. First we clear all existing annotations in the MS1 and MS2 dataframes.

In [ ]:
# removes all the previous annotations by deleting the 'annotation' column from the ms1 & ms2 dataframes
ms2lda.remove_all_annotations()

Here we annotate the MS1 peaks, specifying some heuristic rules on the maximum occurrences of the atoms in the elemental formulae.

The set do_rule_8 parameter below adds the following atoms and limit their max no. of occurrences 
(defined in max_occurrences):
- N   max occurrence 1
- C13 max occurrence 1
- F   max occurrence 2
- Cl  max occurrence 2

If you want to restrict sulphur and phosphor atoms too, use this instead and change the numbers

    max_occurrences = {'N':1, 'S': 1, 'P': 1, C13':1, 'F':2, 'Cl':2}
    
*n_stages* defines the number of stages in the annotation. If it's set to 2, then in the first stage, we perform elemental formulae search using the atoms: CHNOPS. After that, in the second stage, any mass in the list with no results will be re-annotated with the addition of C13, F, Cl to the list of atoms to search.

In [ ]:
max_occurrences = {'N':6, 'S': 2, 'P': 2, 'C13':1, 'F':0, 'Cl':0}
n_stages = 2
tol = 3

ms2lda.annotate_peaks(mode='pos', target='ms1', ppm=5, max_mass=250, 
                      rule_8_max_occurrences=max_occurrences, n_stages=n_stages)

In [ ]:
display(ms2lda.ms1)

We can also annotate the MS2 fragments/losses by specifying different *target* parameter. In the example below, we annotate the MS2 fragment while specifying a conditional mass tolerance to be used in the search. 

We allow the *ppm* parameter to be specified as either fixed tolerance, e.g. in this form:

    tol = 5

or as conditional tolerances as shown in the example below, which for mass <= 80, we use a tolerance of 5 ppm while for mass > 80 and <= 400, we use a tolerance of 10 ppm.

    tol = [(80, 5), (400, 10)]

In [ ]:
max_occurrences = {'N':6, 'S': 2, 'P': 2, 'C13':1, 'F':0, 'Cl':0}
n_stages = 1
tol = [(70, 10), (200, 5)]

# annotate the elemental formulae of MS2 fragments
ms2lda.annotate_peaks(mode='pos', target='ms2_fragment', ppm=tol, max_mass=200, 
                      rule_8_max_occurrences=max_occurrences, n_stages=n_stages)

In [ ]:
display(ms2lda.ms2)

We can also annotate the neutral losses. Set the *target* parameter to 'ms2_loss' and *mode* to 'none'

In [ ]:
# we can also annotate the neutral losses
ms2lda.annotate_peaks(mode='none', target='ms2_loss', ppm=5, max_mass=200, n_stages=1)

In [ ]:
display(ms2lda.ms2)

<h3>d. (Optional) Saving Project</h3>

Save the whole project so we don't have to re-run everything the next time ..

In [ ]:
# leave the message parameter out if nothing to say
ms2lda.save_project('results/beer3pos2.project', message="Beer3Pos analysis for the manuscript with SIRIUS EF Annotation")

<hr/>

<h2>3. Results</h2>

<h3>a. Resuming Project (Optional)</h3>

If you saved the project in step (2c), you can resume from here the next time you load this notebook ..

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = '../'
sys.path.append(basedir)

import numpy as np
import pandas as pd
import pylab as plt
from IPython.display import display
from lda_for_fragments import Ms2Lda

In [2]:
ms2lda = Ms2Lda.resume_from('results/beer3pos.project')

Project loaded from results/beer3pos.project time taken = 26.7905960083
 - input_filenames = 
	../input/manuscript/Beer3pos_MS1filter_Method3_fragments.csv
	../input/manuscript/Beer3pos_MS1filter_Method3_losses.csv
	../input/manuscript/Beer3pos_MS1filter_Method3_ms1.csv
	../input/manuscript/Beer3pos_MS1filter_Method3_ms2.csv
 - df.shape = (1422, 4496)
 - K = 300
 - alpha = 0.166666666667
 - beta = 0.1
 - number of samples stored = 1
 - last_saved_timestamp = Fri Oct 16 02:56:11 2015
 - message = Beer3Pos analysis for the manuscript with SIRIUS EF Annotation


In [3]:
display(ms2lda.ms1)

peakID  MSnParentPeakID  msLevel        rt           mz      intensity  \
1           1                0        1   578.503    70.065165  16431632.0000   
5           5                0        1   566.043    72.080799   1067314.7500   
12         12                0        1  1210.110    72.080818   1025769.1250   
15         15                0        1   468.470    73.064788    925079.4375   
19         19                0        1   656.240    76.039316   1047793.6875   
24         24                0        1  1027.660    76.075699   1636355.7500   
33         33                0        1   562.308    81.033514    766053.0000   
37         37                0        1   632.557    83.060245    705789.6250   
42         42                0        1   486.476    84.044375   2827970.7500   
46         46                0        1   566.043    84.080711    978819.3750   
51         51                0        1   667.614    85.028372   3757659.5000   
55         55                0        1   465.819    85.028381   2102897.0000   
67         67                0        1   415.512    86.059962   1211292.7500   
70         70                0        1   646.272    86.060008   8953359.0000   
78         78                0        1  1005.220    86.060024   1433368.6250   
82         82                0        1  1018.270    86.060026    532986.6250   
86         86                0        1   280.189    86.096375   1282799.1250   
93         93                0        1   519.739    86.096417   2307827.7500   
99         99                0        1   646.272    87.044055  12338473.0000   
105       105                0        1   636.289    87.055276    735020.5000   
111       111                0        1   434.590    89.059678    787915.5625   
123       123                0        1   908.712    89.070924   1793272.5000   
131       131                0        1   645.032    90.054941   2888840.5000   
138       138                0        1   433.349    90.054973    695324.8750   
141       141                0        1   631.311    90.054973   6755740.0000   
148       148                0        1   804.705    90.091364   2163393.0000   
171       171                0        1   402.687    96.044368   1281763.5000   
176       176                0        1   389.139    96.044381   1998793.8750   
185       185                0        1   667.614    97.028420   5013418.0000   
191       191                0        1   467.234    97.028429   1216976.8750   
...       ...              ...      ...       ...          ...            ...   
34244   34244                0        1   270.932   628.382780    780065.3750   
34263   34263                0        1   690.253   633.271325    587176.0000   
34277   34277                0        1   217.007   647.458533   1605510.2500   
34294   34294                0        1   677.736   649.218679   1740094.1250   
34311   34311                0        1   270.932   650.396089    699251.7500   
34325   34325                0        1   223.381   654.332121   1803081.5000   
34352   34352                0        1   273.606   660.523315    538620.6250   
34357   34357                0        1   218.245   663.453627   3320170.0000   
34381   34381                0        1   218.245   666.512338    745498.8125   
34391   34391                0        1   225.864   666.512400    731283.6250   
34401   34401                0        1   666.373   667.228500   2478907.0000   
34430   34430                0        1   676.494   667.228943   4032921.7500   
34459   34459                0        1   661.235   668.260475    804232.2500   
34478   34478                0        1   269.685   672.409058    618715.6250   
34489   34489                0        1   219.484   680.480234   2931464.5000   
34510   34510                0        1   677.736   684.255481  21881570.0000   
34556   34556                0        1   677.736   689.211029   5848416.0000   
34565   34565                0        

In [4]:
display(ms2lda.ms2)

peakID  MSnParentPeakID  msLevel       rt          mz  intensity  \
792       792              789        2  577.268   70.065056   1.000000   
942       942              932        2  531.059  118.086075   1.000000   
17512   17512            17496        2  604.701  104.107330   1.000000   
307       307              298        2  933.275  104.107353   1.000000   
2648     2648             2632        2  495.514  136.062347   1.000000   
936       936              932        2  531.059   58.065453   0.533076   
14567   14567            14559        2  531.059  118.086113   1.000000   
303       303              298        2  933.275   60.080959   0.485118   
2916     2916             2888        2  545.999  138.054489   1.000000   
937       937              932        2  531.059   59.073425   0.344497   
17509   17509            17496        2  604.701   86.096504   0.339775   
17514   17514            17496        2  604.701  125.000565   0.298987   
17511   17511            17496        2  604.701   98.983826   0.237496   
1801     1801             1795        2  484.002   84.044189   1.000000   
797       797              789        2  577.268  116.071083   0.086414   
17503   17503            17496        2  604.701   60.081024   0.190594   
1889     1889             1878        2  566.043   84.080688   1.000000   
1729     1729             1718        2  631.311   83.060341   1.000000   
1756     1756             1745        2  627.569   83.060341   1.000000   
1321     1321             1309        2  420.520  108.044136   1.000000   
18426   18426            18367        2  635.046  248.112946   1.000000   
17506   17506            17496        2  604.701   71.073257   0.148102   
2647     2647             2632        2  495.514  119.035172   0.225609   
17520   17520            17496        2  604.701  258.109772   0.125070   
855       855              848        2  577.268   71.068779   1.000000   
867       867              860        2  577.268   71.068779   1.000000   
2909     2909             2888        2  545.999   94.064751   0.258640   
14560   14560            14559        2  531.059   58.065567   0.204514   
2912     2912             2888        2  545.999  110.059761   0.207781   
301       301              298        2  933.275   58.065456   0.092595   
...       ...              ...      ...      ...         ...        ...   
34641   34641            34618        2  686.519  216.278397   0.001974   
34665   34665            34652        2  689.007  321.063751   0.011014   
30036   30036            30011        2  212.020  119.048721   0.029921   
30108   30108            30083        2  208.313  119.048721   0.029921   
32857   32857            32849        2  956.847  276.134827   0.052007   
27139   27139            27094        2  212.020  151.040359   0.039023   
27345   27345            27300        2  205.736  151.040359   0.039023   
32462   32462            32448        2  563.553  106.065033   0.035201   
32759   32759            32750        2  539.755  105.033417   0.056723   
587       587              583        2  642.551  112.039764   0.006248   
3171     3171             3170        2  559.742   55.054649   0.107236   
19769   19769            19736        2  478.834   95.085373   0.011279   
19843   19843            19810        2  467.234   95.085373   0.011279   
19947   19947            19914        2  470.943   95.085373   0.011279   
20021   20021            19988        2  457.122   95.085373   0.011279   
28285   28285            28253        2  249.534  137.059799   0.004444   
1408     1408             1398        2  467.234  109.059158   0.091647   
33261   33261            33259        2  559.742   57.033623   0.003580   
6208     6208             6195        2  374.457   96.080780   0.040377   
34132   34132            34118        2  659.996  128.070206   0.023226   
32458   32458            32448        2  563.553   91.054131   0.035151   
19740   19740            19736  

<h3>b. Thresholding</h3>

For the purpose of visualisation only, we threshold the document-topic and topic-word distributions produced by LDA, so we can say which topics are used in which documents, and which words 'belongs' to a topic. This needs to be done before step (b) and (c) below.

In [5]:
# Thresholding the doc_topic and topic_word matrices
ms2lda.do_thresholding(th_doc_topic=0.05, th_topic_word=0.01)

From this point onwards, we will refer to an LDA topic as **Mass2Motif** when interpreting the results.

<h3>c. Print Results</h3>

Print which fragment/loss features occur with probability above the thresholds defined above in each Mass2Motif.

In [ ]:
ms2lda.print_motif_features()

We can also save the output to CSV files

In [ ]:
ms2lda.write_results('beer3pos_csv_out')

<h3>d. Cosine Clustering (optional)</h3>

We plot the cosine clustering of the parent ions to investigate the agreement/difference between peaks that have been clustered by cosine clustering vs motifs from LDA. First we need to construct the clustering. To do this, we can use either the hierarchical clustering with Euclidean distance from scipy or an alternative greedy clustering method we devise, which works as follows: **(1)** find the next unprocessed parent ion having max intensity, **(2)** group this to other parents with cosine similarity over the threshold (0.55), **(3)** repeat until all parents have been processed.


In [ ]:
# method is either 'hierarchical' or 'greedy'
# peak_names, clustering = ms2lda.run_cosine_clustering(method='hierarchical', th_clustering=0.80)
peak_names, clustering = ms2lda.run_cosine_clustering(method='greedy', th_clustering=0.55)

In [ ]:
print "Found {} clusters".format(np.max(clustering))

Then we pick some Mass2Motifs, e.g. 102 and 220 below, and plot how all the parent ions cluster based on their cosine similarity. The parent ions of interest that have been assigned to the motif (above the threshold) are indicated as red dots in the plot.

In [ ]:
G, cluster_interests = ms2lda.plot_cosine_clustering(102, clustering, peak_names)
for cluster in sorted(cluster_interests):
    interest_members = cluster_interests[cluster]
    print "Cluster %d with %d parent ion(s) of interest:" % (cluster, len(interest_members))
    display(ms2lda.ms1[ms2lda.ms1['peakID'].isin(interest_members)])

In [ ]:
G, cluster_interests = ms2lda.plot_cosine_clustering(220, clustering, peak_names)
for cluster in sorted(cluster_interests):
    interest_members = cluster_interests[cluster]
    print "Cluster %d with %d parent ion(s) of interest:" % (cluster, len(interest_members))
    display(ms2lda.ms1[ms2lda.ms1['peakID'].isin(interest_members)])

<h3>e. Visualisation</h3>

A visualisation module is provided to explore the results. This can be run in either interactively in the browser or non-interactively by plotting all results in this notebook (which can be a lot of plots!)

<h4>Set Visualisation Parameters</h4>

In [6]:
# If True, an interactive visualisation is shown in a separate tab. 
# You need to interrupt the kernel to stop it once you're done with it (from the menu above, Kernel > Interrupt).
interactive=True

In [7]:
# Used for graph visualisation in the interactive mode only. 
# Specifies the 'special' nodes to be coloured differently.
special_nodes = [
    # you can colour the MS1 peak in the graph
    # 'doc_peakid', where peakid is the peak ID of the MS1 peak    
    ('doc_21758', 'gold'),
    # you can also colour the Mass2Motif in the graph
    ('motif_0', '#ff1493')
]

# If nothing ..
special_nodes = [
    ('motif_19','#0000FF'),
    ('motif_58','#0000FF'),
    ]

In [8]:
# read the annotation assigned to each Mass2Motif from a CSV file
# this could also be stored in e.g. a database
import csv
motif_annotation = {}
for item in csv.reader(open("results/beer3pos_annotation.csv"), skipinitialspace=True):
    key = int(item[0])
    val = item[1]
    print str(key) + '\t' + val
    motif_annotation[key] = val

# here we set all the motifs having annotations as special nodes too
motif_colour = '#CC0000'
to_add_list = ['motif_' + str(x) for x in motif_annotation.keys()]
for item in to_add_list:
    special_nodes.append((item, motif_colour))

# If nothing ..
# motif_annotation = {} # or just leave the 'additional_info' parameter out when calling plot_lda_fragments below

260	Water loss indicative of a free hydroxyl group - often seen in sugary structures
262	Carboxylic acid group (COOH) - generic substructure in amino acids and organic acids
226	Loss of [hexose-H2O] - suggests hexose conjugation (e.g. glucose) substructure
158	Leucine related substructure
243	Conjugation of a phosphate group (H4O4P) substructure
127	Conjugation of a phosphate group (H4O4P) substructure
174	Pyroglutamic acid (pyroglutamate) substructure
59	Pyroglutamic acid (pyroglutamate) substructure
214	Amine loss,  suggests free NH2 group in fragmented molecule
60	Double water loss for metabolites containing OH groups + aliphatic chain,  e.g. sugars
151	[proline-H2O],  suggests conjugated proline substructure.
40	Imidazole group linked to a carboxylgroup through one CH2 group
284	Suggests dihydroxylated benzene ring substructure
276	Alkyl aromatic substructure,  suggests aromatic ring with 2-carbon alkyl chain attached
45	Pipecolic acid (pipecolate) substructure
78	Trimethylated ami

<h4>Run Visualisation</h4>

In [ ]:
ms2lda.plot_lda_fragments(interactive=interactive, to_highlight=special_nodes, additional_info=motif_annotation)

Ranking motifs ...
 - Mass2Motif 0 h-index = 2
 - Mass2Motif 1 h-index = 2
 - Mass2Motif 2 h-index = 1
 - Mass2Motif 3 h-index = 2
 - Mass2Motif 4 h-index = 3
 - Mass2Motif 5 h-index = 5
 - Mass2Motif 6 h-index = 4
 - Mass2Motif 7 h-index = 5
 - Mass2Motif 8 h-index = 3
 - Mass2Motif 9 h-index = 5
 - Mass2Motif 10 h-index = 4
 - Mass2Motif 11 h-index = 4
 - Mass2Motif 12 h-index = 4
 - Mass2Motif 13 h-index = 5
 - Mass2Motif 14 h-index = 2
 - Mass2Motif 15 h-index = 3
 - Mass2Motif 16 h-index = 4
 - Mass2Motif 17 h-index = 4
 - Mass2Motif 18 h-index = 2
 - Mass2Motif 19 h-index = 6
 - Mass2Motif 20 h-index = 3
 - Mass2Motif 21 h-index = 2
 - Mass2Motif 22 h-index = 5
 - Mass2Motif 23 h-index = 7
 - Mass2Motif 24 h-index = 3
 - Mass2Motif 25 h-index = 3
 - Mass2Motif 26 h-index = 6
 - Mass2Motif 27 h-index = 3
 - Mass2Motif 28 h-index = 3
 - Mass2Motif 29 h-index = 1
 - Mass2Motif 30 h-index = 2
 - Mass2Motif 31 h-index = 2
 - Mass2Motif 32 h-index = 2
 - Mass2Motif 33 h-index = 3
 - Ma

In [ ]:
ms2lda.plot_lda_fragments(interactive=interactive)

Ranking motifs ...
 - Mass2Motif 0 h-index = 2
 - Mass2Motif 1 h-index = 2
 - Mass2Motif 2 h-index = 1
 - Mass2Motif 3 h-index = 2
 - Mass2Motif 4 h-index = 3
 - Mass2Motif 5 h-index = 5
 - Mass2Motif 6 h-index = 4
 - Mass2Motif 7 h-index = 5
 - Mass2Motif 8 h-index = 3
 - Mass2Motif 9 h-index = 5
 - Mass2Motif 10 h-index = 4
 - Mass2Motif 11 h-index = 4
 - Mass2Motif 12 h-index = 4
 - Mass2Motif 13 h-index = 5
 - Mass2Motif 14 h-index = 2
 - Mass2Motif 15 h-index = 3
 - Mass2Motif 16 h-index = 4
 - Mass2Motif 17 h-index = 4
 - Mass2Motif 18 h-index = 2
 - Mass2Motif 19 h-index = 6
 - Mass2Motif 20 h-index = 3
 - Mass2Motif 21 h-index = 2
 - Mass2Motif 22 h-index = 5
 - Mass2Motif 23 h-index = 7
 - Mass2Motif 24 h-index = 3
 - Mass2Motif 25 h-index = 3
 - Mass2Motif 26 h-index = 6
 - Mass2Motif 27 h-index = 3
 - Mass2Motif 28 h-index = 3
 - Mass2Motif 29 h-index = 1
 - Mass2Motif 30 h-index = 2
 - Mass2Motif 31 h-index = 2
 - Mass2Motif 32 h-index = 2
 - Mass2Motif 33 h-index = 3
 - Ma